In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import urllib.request
from pulp import *
from scipy import optimize
import math

%matplotlib inline

In [2]:
t = pd.read_csv('../result/slope.csv', index_col=0).T
v = pd.DataFrame(pd.read_excel('../result/lac.xlsx').mean()).T
v.columns = t.columns
slope = pd.concat([t, v])
slope = slope.rename(index={0:'Lactate'})
slope = pd.concat([slope, pd.DataFrame([22, 12.5, 17], columns=['Doubling Time'] , index=slope.columns).T])

In [3]:
slope

,Control,Clone A,Clone B
Glucose,0.084646,0.037859,0.014288
Glutamate,0.017335,-0.004645,0.011940
Lactate,0.636289,3.734983,1.916520
Doubling Time,22.000000,12.500000,17.000000


In [4]:
t = pd.read_csv('../csv/stochastic_matrix.csv', index_col=0).fillna(0)

In [5]:
v = t.T[[
    'R00200', 'R00268', 'R00341', 'R00342', 'R00351',
    'R00405', 'R00658', 'R00703', 'R01015', 'R01049',
    'R01056', 'R01061', 'R01070', 'R01082', 'R01196', 
    'R01197', 'R01325', 'R01512', 'R01518', 'R01528',
    'R01529', 'R01641', 'R01786', 'R01830', 'R01899',
    'R01900', 'R02035', 'R02164', 'R02740', 'R04779'
]].T

for i in v.columns:
    if sum(v[i] == 0) == v.shape[0]:
        v = v.drop(i, axis=1)
        
v['C00007'] = pd.Series(np.zeros(v.shape[0]), index=v.index)
v['C00033'] = pd.Series(np.zeros(v.shape[0]), index=v.index)
v['C00092'] = pd.Series(np.zeros(v.shape[0]), index=v.index)
v['C00014'] = pd.Series(np.zeros(v.shape[0]), index=v.index)
v['C00025'] = pd.Series(np.zeros(v.shape[0]), index=v.index)

# v['C00279'] = pd.Series(np.zeros(v.shape[0]), index=v.index)

In [6]:
df = v.T
df['Glucose_input'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Glucose_input']['C00267'] = 1

df['Oxygen_input'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Oxygen_input']['C00007'] = 1

df['CO2_output'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['CO2_output']['C00011'] = -1

df['Lactate_output'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Lactate_output']['C00186'] = -1

df['Acetate_output'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Acetate_output']['C00033'] = -1

df['Glutamate_input'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Glutamate_input']['C00025'] = 1

df['Ammonia_output'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Ammonia_output']['C00014'] = -1

df['Glu_Exchange'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Glu_Exchange']['C00267'] = -1
df['Glu_Exchange']['C00092'] = 1

df['acetyl_CoA_hydrolase'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['acetyl_CoA_hydrolase']['C00024'] = -1
df['acetyl_CoA_hydrolase']['C00001'] = -1
df['acetyl_CoA_hydrolase']['C00010'] = 1
df['acetyl_CoA_hydrolase']['C00033'] = 1

df['NADP_Exchange'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['NADP_Exchange']['C00005'] = -1
df['NADP_Exchange']['C00003'] = -1
df['NADP_Exchange']['C00006'] = 1
df['NADP_Exchange']['C00004'] = 1

df['ADP_Exchange'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['ADP_Exchange']['C00002'] = -1
df['ADP_Exchange']['C00008'] = 1

df['AMP_Exchange'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['AMP_Exchange']['C00002'] = -1
df['AMP_Exchange']['C00020'] = -1
df['AMP_Exchange']['C00008'] = 2

df['R00835'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['R00835']['C00092'] = -1
df['R00835']['C00006'] = -1
df['R00835']['C01236'] = 1
df['R00835']['C00005'] = 1
df['R00835']['C00080'] = 1

df['R08575'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['R08575']['C05382'] = -1
df['R08575']['C00118'] = -1
df['R08575']['C00279'] = 1
df['R08575']['C00085'] = 1

df['R02164']['C00004'] = -1
df['R02164']['C00003'] = 1
df['R02164']['C15602'] = 0
df['R02164']['C15603'] = 0

df['R00243'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['R00243']['C00025'] = -1
df['R00243']['C00003'] = -1
df['R00243']['C00001'] = -1

df['R00243']['C00026'] = 1
df['R00243']['C00014'] = 1
df['R00243']['C00004'] = 1
df['R00243']['C00080'] = 1

# Oxophos
df['Oxophos'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Oxophos']['C00004'] = -2
df['Oxophos']['C00007'] = -1
df['Oxophos']['C00008'] = -5
df['Oxophos']['C00002'] = 5
df['Oxophos']['C00003'] = 2
df['Oxophos']['C00011'] = 1

In [7]:
BIOMASS = pd.read_csv('../csv/amino_acid.csv',index_col=0)
df_Biomass =  pd.Series(np.zeros(df.shape[0]), index=df.index)

for i in BIOMASS.index:
    df_Biomass[BIOMASS.T[i]['C_num']] = BIOMASS.T[i]['flux']
    
df_Biomass['C00668'] = -0.27519
df_Biomass['C00119'] = (-0.039036) + (-0.036117) + (-0.053446)

In [8]:
df_new = pd.concat([df_Biomass.T, df], axis=1)
df_new = df_new.rename(columns={0: 'BIOMASS'})
df_new = df_new.drop([
    'C00001','C00009','C00080','C00138','C00139','C15602','C15603', 
#     'C00231'
])

In [9]:
def fba(glc, ox):
    x = []
    for i in df_new.columns:
        if (
                (i=='Oxophos') or (i=='R01899') or (i=='R00268') or (i=='R10343') or (i=='R02164') or (i=='R00835') or (i=='R02035') or
                (i=='CO2') or (i == 'Lactate') or (i == 'Acetate') or (i == 'Succinate') or (i == 'Malate') or (i=='ATP_Exchange') or (i=='AMP_Exchange') or
                (i=='Glutamate_output') or (i=='CO2_output') or  (i=='Lactate_output') or (i=='Glutamate_output') or (i=='Ammonia_output')
        ):
            x.append(LpVariable(i, lowBound=0,upBound=10000, cat='Continuous'))

        elif (i=='BIOMASS'):
            x.append(LpVariable(i, lowBound=0, upBound=100, cat='Continuous'))
        
        elif (i=='Oxygen_input'):
            x.append(LpVariable(i, lowBound=0, upBound=ox, cat='Continuous'))
        
        elif (i=='Glucose_input'):
            x.append(LpVariable(i, lowBound=glc, upBound=glc, cat='Continuous'))

        elif (i=='Glutamate_input'):
            x.append(LpVariable(i, lowBound=0, upBound=10, cat='Continuous'))
              
        elif (
            (i=='Succinate_output') or (i=='Malate_output') or (i=='Fumarate_output') or (i=='Acetate_output')
        ):
            x.append(LpVariable(i, lowBound=0, upBound=50, cat='Continuous'))
                
        elif (i=='R00200') or (i=='R01196') or (i=='R01197') or (i=='R00405'):
            x.append(LpVariable(i, lowBound=-10000, upBound=0, cat='Continuous'))

        else:
            x.append(LpVariable(i, lowBound=-10000, upBound=10000, cat='Continuous'))

    m = LpProblem(sense=LpMaximize)
    m += x[0]

    for i in np.dot(df_new, x):
        m += i == 0

    status = m.solve()
    if (LpStatus[status]) == 'Infeasible':
        print(i)
        return

    y = []
    for i in x:
        y.append(i.value())

    j = pd.DataFrame(y, index=[str(i) for i in x]).T
    
    return np.array([j['BIOMASS'][0], j['Lactate_output'][0]])
#                      j['Ammonia_output'][0]])

In [10]:
def f(x):
    x1, x2, x3, y1, y2, y3 = x
    h =[
        fba(x1, y1), 
        fba(x2, y2), 
        fba(x3, y3)    
    ]
    
    if ((h[0]) is None)  or ((h[1]) is None) or ((h[2]) is None):
        return 100
    
    
    p = pd.DataFrame(h, index=['Con', 'CloneA', 'CloneB'], columns=['Biomass', 'Lactate', 
#                                                                     'NH4'
                                                                   ])
    cor = [
        np.corrcoef(p['Biomass'], 
                    [np.log(2)/i for i in slope.T['Doubling Time']]
#         [1/i for i in slope.T['Doubling Time']]
                   )[0, 1],
        np.corrcoef(p['Lactate'], slope.T['Lactate'])[0, 1]
    ]
    
    for i in cor:
        if math.isnan(i):
            return 100
    
    v = 0
    for i in cor:
        if i == i:
            v += i
            
#     print(cor)
    return 2-v

In [11]:
x0 = list(slope.loc['Glucose'].values)+[0.08, 0.015, 0.01]

In [12]:
x0

[0.0846460393271493,
 0.0378590327677735,
 0.014287987036586668,
 0.08,
 0.015,
 0.01]

In [13]:
res1 = optimize.minimize(
    f, 
    x0,
    method='Nelder-Mead'
#     full_output=True
)

Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243
Glutamate_input - R00243


In [14]:
res1

 final_simplex: (array([[1.68160862e-04, 6.76745518e-02, 2.49444380e-02, 4.85845665e-04,
        2.01565027e-02, 7.99143885e-03],
       [1.60632698e-04, 6.76588987e-02, 2.49386013e-02, 4.88251867e-04,
        2.01701799e-02, 7.99035526e-03],
       [1.19316620e-04, 6.77602923e-02, 2.49752925e-02, 4.88010474e-04,
        2.00962826e-02, 7.99552361e-03],
       [1.67833933e-04, 6.76248787e-02, 2.49263289e-02, 4.88652382e-04,
        2.01959038e-02, 7.98855716e-03],
       [1.07630875e-04, 6.77202306e-02, 2.49595694e-02, 4.86123551e-04,
        2.01320593e-02, 7.99274889e-03],
       [8.14957309e-05, 6.77200814e-02, 2.49588668e-02, 4.87104749e-04,
        2.01372505e-02, 7.99217430e-03],
       [7.59300627e-05, 6.76874285e-02, 2.49464610e-02, 4.86766754e-04,
        2.01650022e-02, 7.99008480e-03]]), array([0.00149033, 0.00149119, 0.00149185, 0.00149187, 0.00149193,
       0.00149276, 0.00149336]))
           fun: 0.0014903311408929198
       message: 'Optimization terminated successfull

In [15]:
x1, x2, x3, y1, y2, y3 = res1.x

In [16]:
def fba_result(glc, ox):
    x = []
    for i in df_new.columns:
        if (
                (i=='Oxophos') or (i=='R01899') or (i=='R00268') or (i=='R10343') or (i=='R02164') or (i=='R00835') or (i=='R02035') or
                (i=='CO2') or (i == 'Lactate') or (i == 'Acetate') or (i == 'Succinate') or (i == 'Malate') or (i=='ATP_Exchange') or (i=='AMP_Exchange') or
                (i=='Glutamate_output') or (i=='CO2_output') or  (i=='Lactate_output') or (i=='Glutamate_output') or (i=='Ammonia_output')
        ):
            x.append(LpVariable(i, lowBound=0,upBound=10000, cat='Continuous'))

        elif (i=='BIOMASS'):
            x.append(LpVariable(i, lowBound=0, upBound=10000, cat='Continuous'))
        
        elif (i=='Oxygen_input'):
            x.append(LpVariable(i, lowBound=ox, upBound=ox, cat='Continuous'))
        
        elif (i=='Glucose_input'):
            x.append(LpVariable(i, lowBound=glc, upBound=glc, cat='Continuous'))

        elif (i=='Glutamate_input'):
            x.append(LpVariable(i, lowBound=0, upBound=1000, cat='Continuous'))
              
        elif (
            (i=='Succinate_output') or (i=='Malate_output') or (i=='Fumarate_output') or (i=='Acetate_output')
        ):
            x.append(LpVariable(i, lowBound=0, upBound=50, cat='Continuous'))
                
        elif (i=='R00200') or (i=='R01196') or (i=='R01197') or (i=='R00405'):
            x.append(LpVariable(i, lowBound=-10000, upBound=0, cat='Continuous'))

        else:
            x.append(LpVariable(i, lowBound=-10000, upBound=10000, cat='Continuous'))

    m = LpProblem(sense=LpMaximize)
    m += x[0]

    for i in np.dot(df_new, x):
        m += i == 0

    status = m.solve()
    if (LpStatus[status]) == 'Infeasible':
        print(i)
        return

    y = []
    for i in x:
        y.append(i.value())

    j = pd.DataFrame(y, index=[str(i) for i in x]).T
    
    return j

In [17]:
result = pd.concat([fba_result(x1, y1).T, fba_result(x2, y2).T, fba_result(x3, y3).T], axis=1)
result.columns = ['Control', 'Clone A', 'Clone B']

In [18]:
result

,Control,Clone A,Clone B
BIOMASS,1.204068e-04,0.004995,0.001981
R00200,-6.088127e-05,-0.123915,-0.045355
R00268,0.000000e+00,0.000000,0.000000
R00341,-7.593007e-05,-0.003157,-0.001252
R00342,4.342795e-05,0.001795,0.000712
R00351,0.000000e+00,0.000000,0.000000
R00405,-1.706149e-07,0.000000,0.000000
R00658,1.368113e-04,0.127072,0.046607
R00703,0.000000e+00,-0.121389,-0.044354
R01015,-1.298647e-04,-0.066086,-0.024315
